In [1]:
# import required libraries
import pandas as pd
import numpy as np

In [ ]:
# read excel into dataframe
df_Cardiorespiratory = pd.read_excel("V02Max_Combined.xlsx",sheet_name=None)
# read each sheet and store it in a dataframe
df_Kayaking = df_Cardiorespiratory["Kayaking"]
df_Athletes = df_Cardiorespiratory["Athletes"]
df_Triathlon = df_Cardiorespiratory["Triathlon"]
df_Fencing = df_Cardiorespiratory["Fencing"]

C:\Users\avina\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
C:\Users\avina\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
